# **Import Required Libraries**

In [32]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [33]:
# Install necessary libraries
!pip install transformers accelerate opencv-python matplotlib


# **Downloading Sapiens Depth and Pose Models from Hugging Face**

In [4]:
!wget https://huggingface.co/facebook/sapiens-depth-1b-torchscript/resolve/main/sapiens_1b_render_people_epoch_88_torchscript.pt2 -O sapiens-depth-1b.pt

--2025-02-14 18:44:03--  https://huggingface.co/facebook/sapiens-depth-1b-torchscript/resolve/main/sapiens_1b_render_people_epoch_88_torchscript.pt2
Resolving huggingface.co (huggingface.co)... 18.244.202.60, 18.244.202.68, 18.244.202.118, ...
Connecting to huggingface.co (huggingface.co)|18.244.202.60|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/a7/10/a710e69e3dd1d4c10d5b4b2e132ca47b78cfb2dd0e63648ceabc95ab8d4fc5f3/4ff0c7a8fa48f1d30f97a49aee05abb905f64ee4fe6a35efa805821be5756a8c?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27sapiens_1b_render_people_epoch_88_torchscript.pt2%3B+filename%3D%22sapiens_1b_render_people_epoch_88_torchscript.pt2%22%3B&Expires=1739562244&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczOTU2MjI0NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2E3LzEwL2E3MTBlNjllM2RkMWQ0YzEwZDViNGIyZTEzMmNhNDdiNzhjZmIyZGQwZTYzNjQ4Y2VhYmM5N

# **Model Initilization**




In [5]:
!wget https://huggingface.co/facebook/sapiens-pose-1b-torchscript/resolve/main/sapiens_1b_goliath_best_goliath_AP_639_torchscript.pt2 -O sapiens-pose-1b.pt


--2025-02-14 18:45:55--  https://huggingface.co/facebook/sapiens-pose-1b-torchscript/resolve/main/sapiens_1b_goliath_best_goliath_AP_639_torchscript.pt2
Resolving huggingface.co (huggingface.co)... 18.244.202.73, 18.244.202.118, 18.244.202.68, ...
Connecting to huggingface.co (huggingface.co)|18.244.202.73|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/58/78/5878ffb4f3c9a6e45887705f398a57f2b5e78cdb65e7687bfd112112c53de0f9/d1077c15e66fb2798893e569f71cb063f0b0778af2a62e3c0233807a351c31f3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27sapiens_1b_goliath_best_goliath_AP_639_torchscript.pt2%3B+filename%3D%22sapiens_1b_goliath_best_goliath_AP_639_torchscript.pt2%22%3B&Expires=1739562356&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczOTU2MjM1Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzU4Lzc4LzU4NzhmZmI0ZjNjOWE2ZTQ1ODg3NzA1ZjM5OGE1N2YyYjVlNzhjZGI2NWU

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [7]:
import torch
from transformers import AutoImageProcessor, AutoModelForUniversalSegmentation

# Load Sapiens Pose Model & move to GPU
pose_model = torch.jit.load('/content/sapiens-pose-1b.pt').to(device).half()
pose_model.eval()

# Load Sapiens Depth Model & move to GPU
depth_model = torch.jit.load('/content/sapiens-depth-1b.pt').to(device).half()
depth_model.eval()


RecursiveScriptModule(
  original_name=DepthEstimator
  (data_preprocessor): RecursiveScriptModule(original_name=SegDataPreProcessor)
  (backbone): RecursiveScriptModule(
    original_name=VisionTransformer
    (patch_embed): RecursiveScriptModule(
      original_name=PatchEmbed
      (adaptive_padding): RecursiveScriptModule(original_name=AdaptivePadding)
      (projection): RecursiveScriptModule(original_name=Conv2d)
    )
    (drop_after_pos): RecursiveScriptModule(original_name=Dropout)
    (layers): RecursiveScriptModule(
      original_name=ModuleList
      (0): RecursiveScriptModule(
        original_name=TransformerEncoderLayer
        (ln1): RecursiveScriptModule(original_name=LayerNorm)
        (attn): RecursiveScriptModule(
          original_name=MultiheadAttention
          (qkv): RecursiveScriptModule(original_name=Linear)
          (proj): RecursiveScriptModule(original_name=Linear)
          (proj_drop): RecursiveScriptModule(original_name=Dropout)
          (out_drop):

In [8]:
video_path = '/content/WhatsApp Video 2025-02-14 at 8.14.03 PM.mp4'
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Select start frame, a couple of middle frames, and the last frame
selected_indices = [0, total_frames // 4, total_frames // 2, (3 * total_frames) // 4, total_frames - 1]
frames = []

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if frame_idx in selected_indices:
        frames.append(frame)
    frame_idx += 1

cap.release()


In [9]:
def extract_landmark_coordinates(landmark_tensor, landmark_idx):
    """
    Extracts the (x, y) coordinates of the most confident point in a heatmap.
    If multiple peaks exist, it selects the second-highest peak if the highest one is used.
    """
    heatmap = landmark_tensor[landmark_idx]  # Extract heatmap for this landmark
    flat_index = np.argsort(heatmap.ravel())[::-1]  # Sort in descending order
    y1, x1 = np.unravel_index(flat_index[0], heatmap.shape)  # First highest peak
    y2, x2 = np.unravel_index(flat_index[1], heatmap.shape)  # Second highest peak

    return np.array([x1, y1]), np.array([x2, y2])  # Return two different peaks

In [10]:
def process_frame_with_sapiens_pose(frame):
    """
    Process a frame through Sapiens Pose model and extract (x, y) coordinates.
    """
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image, (1024, 768))  # Resize to match model input
    image_tensor = torch.from_numpy(image_resized).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    image_tensor = image_tensor.half().to('cuda')  # Convert to FP16 and move to GPU

    with torch.no_grad():
        pose_output = pose_model(image_tensor)

    landmarks = pose_output[0].cpu().numpy()  # Convert to NumPy
    return landmarks

In [11]:
def process_frame_with_sapiens_depth(frame):
    """
    Process a frame through Sapiens Depth model to get a depth map.
    """
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image, (1024, 768))  # Resize to match model input
    image_tensor = torch.from_numpy(image_resized).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    image_tensor = image_tensor.half().to('cuda')  # Convert to FP16 and move to GPU

    with torch.no_grad():
        depth_map = depth_model(image_tensor)[0, 0].cpu().numpy()  # Extract depth

    return depth_map

In [12]:
def calculate_3d_distance(p1, p2):
    """Calculate Euclidean distance between two 3D points."""
    return np.linalg.norm(np.array(p1) - np.array(p2))

In [21]:
for idx, frame in enumerate(frames):
    landmarks = process_frame_with_sapiens_pose(frame)

    print(f"\nFrame {idx} - Full Landmark Coordinates:")
    for i, landmark in enumerate(landmarks):
        print(f"Keypoint {i}: ({landmark[0]}, {landmark[1]})")

Streaming output truncated to the last 5000 lines.
  1.3208e-04 -3.9053e-04 -3.2330e-04  5.0068e-04  3.5214e-04 -2.6131e-04
 -1.6093e-04  6.3467e-04  4.8923e-04 -3.6669e-04 -1.9526e-04  4.5419e-04
  2.3293e-04 -4.2152e-04 -2.4295e-04  3.3307e-04  2.6047e-05 -4.7970e-04
 -4.1628e-04  2.7323e-04 -6.7711e-05 -5.4646e-04 -5.6982e-04  2.6965e-04
 -3.1829e-05 -5.7983e-04 -6.1083e-04  2.6655e-04 -2.2769e-05 -6.3992e-04
 -6.2132e-04  2.6155e-04 -9.2864e-05 -6.3801e-04 -5.7745e-04  2.7919e-04
 -8.3447e-05 -6.0511e-04 -5.8794e-04  3.4404e-04 -1.7881e-05 -5.7030e-04
 -6.2990e-04  4.3082e-04  4.0114e-05 -6.1369e-04 -6.1560e-04  3.4618e-04
 -6.9737e-05 -5.9652e-04 -6.3372e-04  1.8954e-04 -3.4237e-04 -5.0116e-04
 -7.3433e-04  1.6129e-04 -5.1117e-04 -7.4625e-04 -1.3437e-03 -3.2616e-04
 -9.7752e-04 -2.1744e-04 -5.2452e-04  5.6362e-04  2.1219e-04 -4.9472e-05
  8.2970e-04  1.7948e-03  1.6375e-03  4.5204e-04  1.7052e-03  2.4815e-03
  2.2373e-03  6.0987e-04  1.7843e-03  2.7027e-03  2.4357e-03  8.2207e-04


In [45]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 31.1 MB/s eta 0:00:00


In [52]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=2, smooth_landmarks=True)

# Function to compute Euclidean 3D distance
def calculate_3d_distance(point1, point2):
    """ Compute the Euclidean distance between two 3D points. """
    return np.linalg.norm(np.array(point1) - np.array(point2))

# List to store distances across all frames
distances = []

for idx, frame in enumerate(frames):
    # Convert frame to RGB (Mediapipe expects RGB format)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Run Mediapipe Pose model
    results = pose.process(frame_rgb)

    if not results.pose_landmarks:
        print(f"Frame {idx}: No pose detected.")
        continue

    # Extract landmarks for left and right ankles
    left_ankle = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE]
    right_ankle = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE]

    # Convert normalized coordinates to image space
    frame_h, frame_w = frame.shape[:2]
    left_x, left_y = int(left_ankle.x * frame_w), int(left_ankle.y * frame_h)
    right_x, right_y = int(right_ankle.x * frame_w), int(right_ankle.y * frame_h)

    # Get depth map from Meta Sapiens
    depth_map = process_frame_with_sapiens_depth(frame)

    # Ensure coordinates are within depth map bounds
    h, w = depth_map.shape
    left_x, left_y = np.clip(left_x, 0, w - 1), np.clip(left_y, 0, h - 1)
    right_x, right_y = np.clip(right_x, 0, w - 1), np.clip(right_y, 0, h - 1)

    # Extract depth values for each ankle
    left_ankle_depth = depth_map[left_y, left_x]
    right_ankle_depth = depth_map[right_y, right_x]

    # Print raw depth values
    print(f"Frame {idx}: Left Ankle Depth = {left_ankle_depth:.3f} meters")
    print(f"Frame {idx}: Right Ankle Depth = {right_ankle_depth:.3f} meters")

    # Construct 3D points (X, Y, Depth)
    left_ankle_3d = (left_x, left_y, left_ankle_depth)
    right_ankle_3d = (right_x, right_y, right_ankle_depth)

    # Compute 3D distance (No Clipping)
    distance_between_feet = calculate_3d_distance(left_ankle_3d, right_ankle_3d)

    print(f"Frame {idx}: Distance Between Feet = {distance_between_feet:.3f} meters")

    # Store the computed distance
    distances.append(distance_between_feet)


print("\nFinal Distances Across Frames:")
print(distances)

Frame 0: Left Ankle Depth = 0.364 meters
Frame 0: Right Ankle Depth = 0.313 meters
Frame 0: Distance Between Feet = 298.000 meters
Frame 1: Left Ankle Depth = 0.415 meters
Frame 1: Right Ankle Depth = 0.348 meters
Frame 1: Distance Between Feet = 177.000 meters
Frame 2: Left Ankle Depth = 0.379 meters
Frame 2: Right Ankle Depth = 0.431 meters
Frame 2: Distance Between Feet = 73.000 meters
Frame 3: Left Ankle Depth = 0.562 meters
Frame 3: Right Ankle Depth = 0.477 meters
Frame 3: Distance Between Feet = 97.000 meters
Frame 4: Left Ankle Depth = 0.307 meters
Frame 4: Right Ankle Depth = 0.313 meters
Frame 4: Distance Between Feet = 218.000 meters

Final Distances Across Frames:
[298.0000042852339, 177.00001264089875, 73.00001834843026, 97.00003720803875, 218.00000008544245]
